In [1]:
#load in required packages
import mediapipe as mp #mediapipe
import cv2 #opencv
import math #basic operations
import numpy as np #basic operations
import pandas as pd #data wrangling
import csv #csv saving
import os #some basic functions for inspecting folder structure etc.

#list all videos in input_videofolder
from os import listdir
from os.path import isfile, join
mypath = "./input_videos/" #this is your folder with (all) your video(s)
#time series output folder
inputfol = "./input_videos/"
outputf_mask = "./Output_Videos/"
outtputf_ts = "./Output_TimeSeries/"
# create output folders if they do not exist
if not os.path.exists(outputf_mask):
    os.makedirs(outputf_mask)
if not os.path.exists(outtputf_ts):
    os.makedirs(outtputf_ts)

#check videos to be processed
print("The following folder is set as the output folder where all the pose time series are stored")
print(os.path.abspath(outtputf_ts))
print("\n The following folder is set as the output folder for saving the masked videos ")
print(os.path.abspath(outputf_mask))
print("\n The following video(s) will be processed for masking: ")


The following folder is set as the output folder where all the pose time series are stored
/Users/esagha/Projects/medal_workshop_on_multimodal_interaction/Pose_Estimation_Mediapipe_Tutorial/Scripts/Output_TimeSeries

 The following folder is set as the output folder for saving the masked videos 
/Users/esagha/Projects/medal_workshop_on_multimodal_interaction/Pose_Estimation_Mediapipe_Tutorial/Scripts/Output_Videos

 The following video(s) will be processed for masking: 


In [2]:
import glob

# This will return a list of all .mp4 paths matching the pattern
vfiles = glob.glob(
    "./input_videos/*.mp4"
)
# remove files that end with "overview.mp4"
# vfiles = [f for f in vfiles if not f.endswith("overview.mp4")]
# # keep only pair04_synced_ppB
# vfiles = [f for f in vfiles if "pair04_synced_ppB" in f]
print(vfiles)


['./input_videos/zHELICOPTER.mp4', './input_videos/zBICYCLE_FIETS.mp4']


In [3]:
vfiles

['./input_videos/zHELICOPTER.mp4', './input_videos/zBICYCLE_FIETS.mp4']

In [4]:
#initialize modules and functions

#load in mediapipe modules
mp_holistic = mp.solutions.holistic
# Import drawing_utils and drawing_styles.
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
 
##################FUNCTIONS AND OTHER VARIABLES
#landmarks 33x that are used by Mediapipe (Blazepose)
markersbody = ['NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_OUTER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
          'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW', 
          'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX',
          'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE',
          'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX']
costume_markers = ['LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW', 
          'RIGHT_ELBOW']
markershands = ['LEFT_WRIST', 'LEFT_THUMB_CMC', 'LEFT_THUMB_MCP', 'LEFT_THUMB_IP', 'LEFT_THUMB_TIP', 'LEFT_INDEX_FINGER_MCP',
              'LEFT_INDEX_FINGER_PIP', 'LEFT_INDEX_FINGER_DIP', 'LEFT_INDEX_FINGER_TIP', 'LEFT_MIDDLE_FINGER_MCP', 
               'LEFT_MIDDLE_FINGER_PIP', 'LEFT_MIDDLE_FINGER_DIP', 'LEFT_MIDDLE_FINGER_TIP', 'LEFT_RING_FINGER_MCP', 
               'LEFT_RING_FINGER_PIP', 'LEFT_RING_FINGER_DIP', 'LEFT_RING_FINGER_TIP', 'LEFT_PINKY_FINGER_MCP', 
               'LEFT_PINKY_FINGER_PIP', 'LEFT_PINKY_FINGER_DIP', 'LEFT_PINKY_FINGER_TIP',
              'RIGHT_WRIST', 'RIGHT_THUMB_CMC', 'RIGHT_THUMB_MCP', 'RIGHT_THUMB_IP', 'RIGHT_THUMB_TIP', 'RIGHT_INDEX_FINGER_MCP',
              'RIGHT_INDEX_FINGER_PIP', 'RIGHT_INDEX_FINGER_DIP', 'RIGHT_INDEX_FINGER_TIP', 'RIGHT_MIDDLE_FINGER_MCP', 
               'RIGHT_MIDDLE_FINGER_PIP', 'RIGHT_MIDDLE_FINGER_DIP', 'RIGHT_MIDDLE_FINGER_TIP', 'RIGHT_RING_FINGER_MCP', 
               'RIGHT_RING_FINGER_PIP', 'RIGHT_RING_FINGER_DIP', 'RIGHT_RING_FINGER_TIP', 'RIGHT_PINKY_FINGER_MCP', 
               'RIGHT_PINKY_FINGER_PIP', 'RIGHT_PINKY_FINGER_DIP', 'RIGHT_PINKY_FINGER_TIP']
facemarks = [str(x) for x in range(478)] #there are 478 points for the face mesh (see google holistic face mesh info for landmarks)

print("Note that we have the following number of pose keypoints for markers body")
print(len(markersbody))

print("\n Note that we have the following number of pose keypoints for markers hands")
print(len(markershands))

print("\n Note that we have the following number of pose keypoints for markers face")
print(len(facemarks ))

#set up the column names and objects for the time series data (add time as the first variable)
markerxyzbody = ['time']
markerxyzhands = ['time']
markerxyzface = ['time']

for mark in markersbody:
    for pos in ['X', 'Y', 'Z', 'visibility']: #for markers of the body you also have a visibility reliability score
        nm = pos + "_" + mark
        markerxyzbody.append(nm)
for mark in markershands:
    for pos in ['X', 'Y', 'Z']:
        nm = pos + "_" + mark
        markerxyzhands.append(nm)
for mark in facemarks:
    for pos in ['X', 'Y', 'Z']:
        nm = pos + "_" + mark
        markerxyzface.append(nm)

#check if there are numbers in a string
def num_there(s):
    return any(i.isdigit() for i in s)

#take some google classification object and convert it into a string
def makegoginto_str(gogobj):
    gogobj = str(gogobj).strip("[]")
    gogobj = gogobj.split("\n")
    return(gogobj[:-1]) #ignore last element as this has nothing

#make the stringifyd position traces into clean numerical values
def listpostions(newsamplemarks):
    newsamplemarks = makegoginto_str(newsamplemarks)
    tracking_p = []
    for value in newsamplemarks:
        if num_there(value):
            stripped = value.split(':', 1)[1]
            stripped = stripped.strip() #remove spaces in the string if present
            tracking_p.append(stripped) #add to this list  
    return(tracking_p)

Note that we have the following number of pose keypoints for markers body
33

 Note that we have the following number of pose keypoints for markers hands
42

 Note that we have the following number of pose keypoints for markers face
478


In [5]:
kp = []
mp_to_coco_body = [0, 2, 5, 7, 8, 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]
mp_to_coco_foot = [
    31, 31, 29,   # left_big_toe, left_small_toe (dup), left_heel
    32, 32, 30    # right_big_toe, right_small_toe (dup), right_heel
]
from mp2dlib import convert_landmarks_mediapipe_to_dlib
# results.face_landmarks is the MP FaceMesh output


In [6]:
vfiles

['./input_videos/zHELICOPTER.mp4', './input_videos/zBICYCLE_FIETS.mp4']

In [13]:
# MASKING AND BLURRING OPTIONS 
skeleton = True
skeleton_face_only = False         # Only show face skeleton (no body, no hands)
whitebackground = False            # Grey background with skeleton only (no body, no face)
maskingbody = False                # Masks the body silhouette with fully black color (original masked-piper approach)
maskingface = False                # Masks the face (fully black color)
blurringface = False               # Blurs the face
blurringbody = True                # Blurs the body region
blurringfactor = 1                 # Blurring intensity (0-1, 1 is full blur)
# TRACE OPTIONS
add_finger_traces = True           # Add fading traces for index fingers
trace_length_seconds = 1.5         # Length of trace in seconds
trace_color_left = (0, 255, 0)     # Green for left index finger trace
trace_color_right = (0, 0, 255)    # Blue for right index finger trace

# Process videos
# MASKING AND BLURRING OPTIONS 
skeleton = True
skeleton_face_only = False         # Only show face skeleton (no body, no hands)
whitebackground = False            # Grey background with skeleton only (no body, no face)
maskingbody = False                # Masks the body silhouette with fully black color (original masked-piper approach)
maskingface = False                # Masks the face (fully black color)
blurringface = False               # Blurs the face
blurringbody = True                # Blurs the body region
blurringfactor = 1                 # Blurring intensity (0-1, 1 is full blur)
# TRACE OPTIONS
add_finger_traces = True           # Add fading traces for index fingers
trace_length_seconds = 2           # Length of trace in seconds
trace_color_left = (0, 255, 0)     # Green for left index finger trace
trace_color_right = (0, 0, 255)    # Blue for right index finger trace

# Process videos
for vidf in vfiles:
    print(f"Processing video: {vidf}")
    print(f"Video {vfiles.index(vidf)+1} of {len(vfiles)}")
    
    videoname = vidf
    videoloc = inputfol + videoname
    capture = cv2.VideoCapture(videoloc)
    frameWidth = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
    frameHeight = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
    samplerate = capture.get(cv2.CAP_PROP_FPS)
    
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(outputf_mask + videoname, fourcc, 
                          fps=samplerate, frameSize=(int(frameWidth), int(frameHeight)))
    
    time = 0
    tsbody = [markerxyzbody]
    tshands = [markerxyzhands]
    tsface = [markerxyzface]
    
    # Initialize trace buffers for index fingers
    if add_finger_traces:
        trace_frames = int(trace_length_seconds * samplerate)
        left_finger_trace = []  # Store (x, y) positions for left index finger
        right_finger_trace = []  # Store (x, y) positions for right index finger
    
    with mp_holistic.Holistic(static_image_mode=False, enable_segmentation=True, refine_face_landmarks=True) as holistic:
        while True:
            ret, image = capture.read()
            if ret == True:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = holistic.process(image)
                h, w, c = image.shape
                
                if np.all(results.face_landmarks) != None:
                    # Apply white background modes first (they override other options)
                    if whitebackground:
                        # Create white background with only landmarks
                        white_image = np.full((h, w, 3), (255, 255, 255), dtype=np.uint8)
                        original_image = cv2.cvtColor(white_image, cv2.COLOR_RGB2BGR)
                    else:
                        # Convert to BGR for further processing
                        original_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                    
                    # Apply body masking if enabled (makes body fully black - original masked-piper)
                    if maskingbody and not whitebackground:
                        # Original masking logic
                        image_with_alpha = np.concatenate([image, np.full((h, w, 1), 255, dtype=np.uint8)], axis=-1)
                        mask_img = np.zeros_like(image, dtype=np.uint8)
                        mask_img[:, :] = (255,255,255)
                        segm_2class = 0.2 + 0.8 * results.segmentation_mask
                        segm_2class = np.repeat(segm_2class[..., np.newaxis], 3, axis=2)
                        annotated_image = mask_img * segm_2class * (1 - segm_2class)
                        mask = np.concatenate([annotated_image, np.full((h, w, 1), 255, dtype=np.uint8)], axis=-1)
                        image_with_alpha[mask==0]=0
                        original_image = cv2.cvtColor(image_with_alpha, cv2.COLOR_RGB2BGR)
                    
                    # Apply blurring to body if enabled
                    if blurringbody and not whitebackground:
                        # Get body mask
                        segm_2class = 0.2 + 0.8 * results.segmentation_mask
                        body_mask = segm_2class
                        
                        kernel_size = int(51 * blurringfactor)
                        if kernel_size % 2 == 0:
                            kernel_size += 1
                        blurred_image = cv2.GaussianBlur(original_image, (kernel_size, kernel_size), 0)
                        
                        # Apply blur only to body region
                        body_mask_3channel = cv2.merge([body_mask] * 3)
                        original_image = (original_image * (1 - body_mask_3channel * blurringfactor) + 
                                        blurred_image * (body_mask_3channel * blurringfactor)).astype(np.uint8)
                    
                    # Apply face blurring if enabled
                    if blurringface and results.face_landmarks and not whitebackground:
                        face_mask = np.zeros((h, w), dtype=np.uint8)
                        landmarks = results.face_landmarks.landmark
                        
                        # Get all face points for a full face mask
                        face_points = np.array([(int(landmarks[i].x * w), int(landmarks[i].y * h)) 
                                              for i in range(len(landmarks))], dtype=np.int32)
                        
                        # Create convex hull of face points
                        hull = cv2.convexHull(face_points)
                        cv2.fillConvexPoly(face_mask, hull, 255)
                        
                        # Blur the face region
                        kernel_size = int(51 * blurringfactor)
                        if kernel_size % 2 == 0:
                            kernel_size += 1
                        blurred_image = cv2.GaussianBlur(original_image, (kernel_size, kernel_size), 0)
                        
                        # Apply blur only to face region
                        face_mask_3channel = cv2.cvtColor(face_mask, cv2.COLOR_GRAY2BGR) / 255.0
                        original_image = (original_image * (1 - face_mask_3channel * blurringfactor) + 
                                        blurred_image * (face_mask_3channel * blurringfactor)).astype(np.uint8)
                    
                    # Apply face masking if enabled (makes face fully black)
                    if maskingface and results.face_landmarks and not whitebackground:
                        face_mask = np.zeros((h, w), dtype=np.uint8)
                        landmarks = results.face_landmarks.landmark
                        
                        # Create mask for entire face
                        face_points = np.array([(int(landmarks[i].x * w), int(landmarks[i].y * h)) 
                                              for i in range(len(landmarks))], dtype=np.int32)
                        
                        # Create convex hull of face points for masking
                        hull = cv2.convexHull(face_points)
                        cv2.fillConvexPoly(face_mask, hull, 255)
                        
                        # Apply masking - make face fully black
                        original_image[face_mask > 0] = (0, 0, 0)  # Make face fully black
                    
                    # Add index finger traces if enabled
                    if add_finger_traces:
                        # Get current index finger positions
                        left_finger_pos = None
                        right_finger_pos = None
                        
                        if results.left_hand_landmarks:
                            # Index finger tip landmark index is 8
                            left_landmark = results.left_hand_landmarks.landmark[8]
                            left_finger_pos = (int(left_landmark.x * w), int(left_landmark.y * h))
                        
                        if results.right_hand_landmarks:
                            # Index finger tip landmark index is 8
                            right_landmark = results.right_hand_landmarks.landmark[8]
                            right_finger_pos = (int(right_landmark.x * w), int(right_landmark.y * h))
                        
                        # Add current positions to trace buffers
                        if left_finger_pos:
                            left_finger_trace.append(left_finger_pos)
                        if right_finger_pos:
                            right_finger_trace.append(right_finger_pos)
                        
                        # Keep trace buffers to specified length
                        if len(left_finger_trace) > trace_frames:
                            left_finger_trace.pop(0)
                        if len(right_finger_trace) > trace_frames:
                            right_finger_trace.pop(0)
                        
                        # Draw fading traces for left index finger
                        for i in range(len(left_finger_trace)-1):
                            if i < len(left_finger_trace) and (i+1) < len(left_finger_trace):
                                # Calculate opacity based on position in trace
                                opacity = int(255 * (i+1) / len(left_finger_trace))
                                alpha = opacity / 255.0
                                
                                # Create a copy for transparency effect
                                overlay = original_image.copy()
                                cv2.line(overlay, left_finger_trace[i], left_finger_trace[i+1], 
                                       trace_color_left, 2)
                                original_image = cv2.addWeighted(overlay, alpha, original_image, 1-alpha, 0)
                        
                        # Draw fading traces for right index finger
                        for i in range(len(right_finger_trace)-1):
                            if i < len(right_finger_trace) and (i+1) < len(right_finger_trace):
                                # Calculate opacity based on position in trace
                                opacity = int(255 * (i+1) / len(right_finger_trace))
                                alpha = opacity / 255.0
                                
                                # Create a copy for transparency effect
                                overlay = original_image.copy()
                                cv2.line(overlay, right_finger_trace[i], right_finger_trace[i+1], 
                                       trace_color_right, 2)
                                original_image = cv2.addWeighted(overlay, alpha, original_image, 1-alpha, 0)
                    
                    # Draw landmarks conditionally
                    if skeleton:
                        mp_drawing.draw_landmarks(original_image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                        mp_drawing.draw_landmarks(original_image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                        
                        mp_drawing.draw_landmarks(
                            original_image,
                            results.face_landmarks,
                            mp_holistic.FACEMESH_TESSELATION,
                            landmark_drawing_spec=None,
                            connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style()
                        )
                        mp_drawing.draw_landmarks(
                            original_image,
                            results.pose_landmarks,
                            mp_holistic.POSE_CONNECTIONS,
                            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
                        )
                    elif skeleton_face_only:
                        # For face-only mode, draw only face landmarks
                        mp_drawing.draw_landmarks(
                            original_image,
                            results.face_landmarks,
                            mp_holistic.FACEMESH_TESSELATION,
                            landmark_drawing_spec=None,
                            connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style()
                        )
                    
                    # Save time series data - FIXED for hand swapping issue
                    samplebody = listpostions(results.pose_landmarks)
                    sampleface = listpostions(results.face_landmarks)
                    
                    # Process hands separately
                    sampleLH = listpostions(results.left_hand_landmarks)
                    sampleRH = listpostions(results.right_hand_landmarks)
                    
                    # Fill empty left hand with placeholders
                    if len(sampleLH) == 0:
                        sampleLH = ["" for x in range(int(len(markerxyzhands)/2))]
                    
                    # Combine hands
                    samplehands = sampleLH + sampleRH
                    
                    # Add time
                    samplebody.insert(0, time)
                    samplehands.insert(0, time)
                    sampleface.insert(0, time)
                    
                    # Append to time series
                    tsbody.append(samplebody)
                    tshands.append(samplehands)
                    tsface.append(sampleface)
                    
                else:
                    original_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                    # Add NaN data
                    samplebody = [np.nan for x in range(len(markerxyzbody)-1)]
                    samplehands = [np.nan for x in range(len(markerxyzhands)-1)]
                    sampleface = [np.nan for x in range(len(markerxyzface)-1)]
                    samplebody.insert(0, time)
                    samplehands.insert(0, time)
                    sampleface.insert(0, time)
                    tsbody.append(samplebody)
                    tshands.append(samplehands)
                    tsface.append(sampleface)
                
                cv2.imshow("resizedimage", original_image)
                out.write(original_image)
                time = time + (1000/samplerate)
                
            if cv2.waitKey(1) == 27:
                break
            if ret == False:
                break
    
    out.release()
    capture.release()
    cv2.destroyAllWindows()
    
    # # Save CSV files
    # filebody = open(outtputf_ts + vidf[:-4] + '_body.csv', 'w+', newline='')
    # with filebody:
    #     write = csv.writer(filebody)
    #     write.writerows(tsbody)
    
    # filehands = open(outtputf_ts + vidf[:-4] + '_hands.csv', 'w+', newline='')
    # with filehands:
    #     write = csv.writer(filehands)
    #     write.writerows(tshands)
    
    # fileface = open(outtputf_ts + vidf[:-4] + '_face.csv', 'w+', newline='')
    # with fileface:
    #     write = csv.writer(fileface)
    #     write.writerows(tsface)

print("Done with processing all folders; go look in your output folders!") 

Processing video: ./input_videos/zHELICOPTER.mp4
Video 1 of 2
Processing video: ./input_videos/zBICYCLE_FIETS.mp4
Video 2 of 2
Done with processing all folders; go look in your output folders!


OpenCV: Couldn't read video stream from file "./input_videos/./input_videos/zHELICOPTER.mp4"
I0000 00:00:1748277959.903393 8441312 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1748277959.961060 8450864 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748277959.969628 8450875 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748277959.970794 8450873 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748277959.970940 8450875 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748277959.971093 8450869 inference_feedback_manage

In [14]:
import cv2
import mediapipe as mp

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

def draw_and_save_custom_landmarks(image, results, skip_pose_ids=None,
                          hand_landmark_style=None,
                          hand_connection_style=None,
                          pose_point_radius=4,
                          pose_point_color=(0,255,0),
                          pose_connection_style=None,
                          connect_hands_to_body=True,
                          arm_connection_color=(255,0,0),
                          arm_connection_thickness=2, 
                          draw=False):
    h, w, _ = image.shape
    """
    Draws all hand landmarks + filtered pose landmarks on `image`.

    Args:
      image:       BGR image to draw onto.
      results:     Holistic.process(...) results.
      skip_pose_ids: set of mp_holistic.PoseLandmark to omit.
      hand_landmark_style, hand_connection_style:
        DrawingSpec for hands (defaults to MP styles).
      pose_point_radius, pose_point_color:
        circle style for filtered pose points.
      pose_connection_style:
        DrawingSpec for pose connections (defaults to green, thickness=2).
    """
    skip_pose_ids = skip_pose_ids or set()
    # default styles
    hand_landmark_style    = hand_landmark_style    or mp_styles.get_default_hand_landmarks_style()
    hand_connection_style  = hand_connection_style  or mp_styles.get_default_hand_connections_style()
    pose_connection_style  = pose_connection_style  or mp_drawing.DrawingSpec(color=pose_point_color, thickness=2)

    # 1) draw **all** hand landmarks
    frame_keypoints = []
    if results.left_hand_landmarks:
        if draw:
            mp_drawing.draw_landmarks(
                image,
                results.left_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=hand_landmark_style,
                connection_drawing_spec=hand_connection_style
            )
        # add left hand keypoints to frame_keypoints
        for idx, lm in enumerate(results.left_hand_landmarks.landmark):
            frame_keypoints.append([lm.x, lm.y, lm.z, lm.visibility])
    else:
        # If no left hand landmarks, add placeholders
        for i in range(21):
            frame_keypoints.append([0, 0, 0, 0])
    
    if results.right_hand_landmarks:
        if draw:
            mp_drawing.draw_landmarks(
                image,
                results.right_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=hand_landmark_style,
                connection_drawing_spec=hand_connection_style
            )
        # add right hand keypoints to frame_keypoints
        for idx, lm in enumerate(results.right_hand_landmarks.landmark):
            frame_keypoints.append([lm.x, lm.y, lm.z, lm.visibility])
    else:
        # If no right hand landmarks, add placeholders
        for i in range(21):
            frame_keypoints.append([0, 0, 0, 0])
    # 2) draw **filtered** pose points & connections
    if results.pose_landmarks:
        h, w, _ = image.shape
        if draw:
            # draw the points (skip any in skip_pose_ids)
            for idx, lm in enumerate(results.pose_landmarks.landmark):
                landmark = mp_holistic.PoseLandmark(idx)
                if landmark in skip_pose_ids:
                    continue
                x, y = int(lm.x * w), int(lm.y * h)
                cv2.circle(image, (x, y), pose_point_radius, pose_point_color, -1)
            

        
        
        # add filtered connections to frame_keypoints
        for idx, lm in enumerate(results.pose_landmarks.landmark):
            if idx in skip_pose_ids:
                continue
            else:
                frame_keypoints.append([lm.x, lm.y, lm.z, lm.visibility])
            

        # draw connections
        if draw:
            # build filtered connections
            filtered_conns = [
                (start, end)
                for (start, end) in mp_holistic.POSE_CONNECTIONS
                if (mp_holistic.PoseLandmark(start) not in skip_pose_ids and
                    mp_holistic.PoseLandmark(end  ) not in skip_pose_ids)
            ]
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                filtered_conns,
                landmark_drawing_spec=None,            # already drew circles
                connection_drawing_spec=pose_connection_style
            )
            # 3) optionally connect each hand’s wrist back to its elbow
            if connect_hands_to_body and results.pose_landmarks:
                # LEFT
                if results.left_hand_landmarks:
                    l_wrist = results.left_hand_landmarks.landmark[0]
                    l_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW]
                    p1 = (int(l_wrist.x * w), int(l_wrist.y * h))
                    p2 = (int(l_elbow.x * w), int(l_elbow.y * h))
                    cv2.line(image, p1, p2, arm_connection_color, arm_connection_thickness)

                # RIGHT
                if results.right_hand_landmarks:
                    r_wrist = results.right_hand_landmarks.landmark[0]
                    r_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW]
                    p1 = (int(r_wrist.x * w), int(r_wrist.y * h))
                    p2 = (int(r_elbow.x * w), int(r_elbow.y * h))
                    cv2.line(image, p1, p2, arm_connection_color, arm_connection_thickness)
    else:   
        # If no pose landmarks, add placeholders
        for i in range(len(costume_markers)):
            frame_keypoints.append([0, 0, 0, 0])

    return image, frame_keypoints


In [18]:
import cv2
import mediapipe as mp
from tqdm import tqdm

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

# Which PoseLandmark indices we want to skip because
SKIP_POSE_IDS = {
    mp_holistic.PoseLandmark.LEFT_WRIST,
    mp_holistic.PoseLandmark.RIGHT_WRIST,
    mp_holistic.PoseLandmark.LEFT_PINKY,
    mp_holistic.PoseLandmark.RIGHT_PINKY,
    mp_holistic.PoseLandmark.LEFT_INDEX,
    mp_holistic.PoseLandmark.RIGHT_INDEX,
    mp_holistic.PoseLandmark.LEFT_THUMB,
    mp_holistic.PoseLandmark.RIGHT_THUMB,
    mp_holistic.PoseLandmark.LEFT_HIP,
    mp_holistic.PoseLandmark.RIGHT_HIP,
    mp_holistic.PoseLandmark.LEFT_KNEE,
    mp_holistic.PoseLandmark.RIGHT_KNEE,
    mp_holistic.PoseLandmark.LEFT_ANKLE,
    mp_holistic.PoseLandmark.RIGHT_ANKLE,
    mp_holistic.PoseLandmark.LEFT_HEEL,
    mp_holistic.PoseLandmark.RIGHT_HEEL,
    mp_holistic.PoseLandmark.LEFT_FOOT_INDEX,
    mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX,
    mp_holistic.PoseLandmark.NOSE,
    mp_holistic.PoseLandmark.LEFT_EYE_INNER,
    mp_holistic.PoseLandmark.LEFT_EYE,
    mp_holistic.PoseLandmark.LEFT_EYE_OUTER,
    mp_holistic.PoseLandmark.RIGHT_EYE_OUTER,
    mp_holistic.PoseLandmark.RIGHT_EYE,
    mp_holistic.PoseLandmark.RIGHT_EYE_INNER,
    mp_holistic.PoseLandmark.RIGHT_EYE_OUTER,
    mp_holistic.PoseLandmark.LEFT_EAR,
    mp_holistic.PoseLandmark.RIGHT_EAR,
    mp_holistic.PoseLandmark.MOUTH_LEFT,
    mp_holistic.PoseLandmark.MOUTH_RIGHT, 
}
# Process videos
for vidf in vfiles:
    print(f"Processing video: {vidf}")
    print(f"Video {vfiles.index(vidf)+1} of {len(vfiles)}")
    
    videoname = vidf
    videoloc = inputfol + videoname
    capture = cv2.VideoCapture(0)
    # get the number of frames in the video
    frameWidth = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
    frameHeight = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
    samplerate = capture.get(cv2.CAP_PROP_FPS)
    # get the number of frames in the video
    num_frames = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    # get the number of frames in the video
    print(f"Number of frames in the video: {num_frames}")
    with mp_holistic.Holistic(static_image_mode=False,           # Video stream mode :contentReference[oaicite:7]{index=7}
    model_complexity=1,                # Highest-accuracy pose model :contentReference[oaicite:8]{index=8}
    refine_face_landmarks=False,        # Finer facial detail (iris, contours) :contentReference[oaicite:9]{index=9}
    enable_segmentation=False,          # Person mask for effects :contentReference[oaicite:10]{index=10}
    smooth_landmarks=True,             # Temporal smoothing to reduce jitter :contentReference[oaicite:11]{index=11}
    min_detection_confidence=0.7,      # Filter weak detections :contentReference[oaicite:12]{index=12}
    min_tracking_confidence=0.7        # Filter unstable tracks :contentReference[oaicite:13]{index=13}
    ) as holistic:
        all_kpts = []
        for i in tqdm(range(num_frames), desc="Processing frames", unit="frame"):
            
            ret, frame = capture.read()
            if not ret:
                break
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(image)
            h, w, _ = image.shape
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            image, kpts = draw_and_save_custom_landmarks(
                    image,
                    results,
                    skip_pose_ids=SKIP_POSE_IDS,
                    pose_point_radius=5,
                    pose_point_color=(0,255,0),
                    connect_hands_to_body=True,
                    arm_connection_color=(255,0,0),       # red lines for the “arm” link
                    arm_connection_thickness=2
                )
            all_kpts.append(kpts)
            if cv2.waitKey(1) == 27:
               break
            cv2.imshow("merged_landmarks", image)
            cv2.waitKey(1)
            capture.release()
            cv2.destroyAllWindows()
    all_kpts = np.array(all_kpts)
    # Save the keypoints as npy array
    video_name = vidf.split('/')[-1].split('.')[0]
    np.save(outtputf_ts + video_name+ '_all_kpts_17.npy', all_kpts)
    # 

I0000 00:00:1748281750.190886 8441312 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro


Processing video: ./input_videos/zHELICOPTER.mp4
Video 1 of 2
Number of frames in the video: 0


Processing frames: 0frame [00:00, ?frame/s]

Processing video: ./input_videos/zBICYCLE_FIETS.mp4
Video 2 of 2
Number of frames in the video: 0



W0000 00:00:1748281750.252812 8490702 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748281750.261634 8490703 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748281750.262815 8490710 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748281750.262826 8490706 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748281750.262840 8490702 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748281750.266227 8490702 inference_feedback_manager.cc:114] Feedback manager

In [10]:
# import glob

# # This will return a list of all .mp4 paths matching the pattern
# file_paths = glob.glob(
#     "/Users/esamghaleb/Documents/ResearchData/CABB Small Dataset/processed_audio_video/*/*.mp4"
# )

# print(file_paths)
